In [1]:
#Importing packages
from transformers import ViTModel, ViTImageProcessor
import torch
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import chromadb
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
#Import library code
import helper_code.dataloading as dataloading
import helper_code.model_functions as model_functions
import helper_code.data_vis as data_vis

d:\Projects\DSC180A-Capstone\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
labels_csv = "camera_data/coronado_hills_binary_11-15-2025.csv"
image_dir = "camera_data/images/"

data = dataloading.get_data(labels_csv=labels_csv, image_dir=image_dir)

In [3]:
train, val, test = dataloading.get_train_val_test(data = data, output_csvs=True)

In [4]:
train_embeddings, train_labels, _, _  = dataloading.load_full_embeddings(train, "q1_train_embeddings")

In [5]:
train_embedding_dataset = dataloading.CustomEmbeddingDataset(train_embeddings, train_labels)
#val_embedding_dataset = dataloading.CustomEmbeddingDataset(embeddings, labels)

train_embedding_dataloader = DataLoader(train_embedding_dataset, batch_size=32, shuffle=True, pin_memory=True)
#val_embedding_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True, pin_memory=True)

In [6]:
device = "cuda"

encoder = model_functions.create_encoder()
encoder.to(device);

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
encoder.load_state_dict(torch.load('weights/model_weights_camera_11-17-25.pth', weights_only=True))
encoder.eval();

In [8]:
classification_head = model_functions.ClassificationHead()
classification_head.to(device)

ClassificationHead(
  (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (head): Sequential(
    (0): Linear(in_features=768, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=2, bias=True)
    (3): Softmax(dim=1)
  )
)

In [9]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classification_head.parameters(), lr=1e-4) # Optimize only the new head

In [10]:
num_epochs = 1
for epoch in range(num_epochs):
    classification_head.train() # Set model to training mode
    for batch in train_embedding_dataloader:
        embeddings = batch['embeddings'].to(device).float()
        labels = batch['labels'].to(device).long()

        optimizer.zero_grad()
        outputs = classification_head(embeddings)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [11]:
classification_head.eval()

acc_sum = 0
for batch in train_embedding_dataloader:
        embeddings = batch['embeddings'].to(device).float()
        labels = batch['labels'].to(device).long()

        outputs = classification_head(embeddings)
        acc_sum += (torch.argmax(outputs, dim = -1) == labels).int().sum()

print(f"Accuracy: {acc_sum / len(train_embedding_dataloader.dataset)}")

Accuracy: 0.9223408699035645


In [ ]:
embeddings_tensor = torch.Tensor(train_embeddings.to_numpy()).to(device)

outputs = classification_head(embeddings_tensor)

acc

tensor([[0.9948, 0.0052],
        [0.0012, 0.9988],
        [0.0011, 0.9989],
        ...,
        [0.9936, 0.0064],
        [0.9932, 0.0068],
        [0.9947, 0.0053]], device='cuda:0', grad_fn=<SoftmaxBackward0>)